In [1]:
from torch import nn
from bound_propagation import BoundModelFactory, HyperRectangle
from bound_propagation.polynomial import Pow
from bound_propagation.bounds import LinearBounds
import torch
import numpy as np
import cvxpy as cp
import gymnasium as gym

(CVXPY) Jan 17 11:46:43 AM: Encountered unexpected exception importing solver SCS:
ImportError("dlopen(/Users/koentuin/Documents/Studie/Minor Engineering wit AI/CAI/venv/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so, 0x0002): tried: '/Users/koentuin/Documents/Studie/Minor Engineering wit AI/CAI/venv/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/koentuin/Documents/Studie/Minor Engineering wit AI/CAI/venv/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so' (no such file), '/Users/koentuin/Documents/Studie/Minor Engineering wit AI/CAI/venv/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))")
(CVXPY) Jan 17 11:46:43 AM: Encountered unexpected exception importing solver OSQP:
ImportError("dlopen(/Users/koentuin/Documents/Studie/

In [2]:
def create_action_partitions(env, partitions):
    """
    Generates partitions of the action space of a reinforcement learning environment into smaller hyperrectangles.

    This function is useful for discretizing a continuous action space into smaller, manageable parts, allowing for more granular exploration or analysis.

    Parameters:
    - env (Environment): The reinforcement learning environment object, which should have an action space attribute.
    - partitions (int): The number of partitions to create along each dimension of the action space.

    Each dimension of the action space is divided into 'partitions' number of equal parts. For an action space with 'n' dimensions, this results in 'partitions^n' total hyperrectangles.

    Returns:
    - A list of HyperRectangle objects, each representing a partition of the action space. Each HyperRectangle object is defined by its lower and upper bounds in the action space.
    """
    action_space = env.action_space
    num_actions = action_space.shape[0]
    action_low = action_space.low
    action_high = action_space.high

    res = []

    def generate_partitions(dimensions, lower, upper, current_partition):
        if dimensions == num_actions:
            # If we've reached the number of dimensions, add the current partition
            res.append(HyperRectangle(np.array(lower), np.array(upper)))
        else:
            # Calculate the size of the partition for the current dimension
            partition_size = (action_high[dimensions] - action_low[dimensions]) / partitions

            for part in range(partitions):
                # Determine the lower and upper bounds for the current dimension
                dim_lower_bound = action_low[dimensions] + part * partition_size
                dim_upper_bound = dim_lower_bound + partition_size

                # Recursively generate partitions for the next dimension
                generate_partitions(dimensions + 1, lower + [dim_lower_bound], upper + [dim_upper_bound], current_partition)

    generate_partitions(0, [], [], [])

    return res

In [3]:
# import unittest
# import numpy as np

# class TestCreateActionPartitions(unittest.TestCase):

#     def setUp(self):
#         # Mocking an environment and HyperRectangle class for testing
#         class MockEnv:
#             class action_space:
#                 low = np.array([0, 0])
#                 high = np.array([10, 10])
#                 shape = (2,)

#         self.MockEnv = MockEnv
#         self.HyperRectangle = HyperRectangle

#     def test_partition_generation(self):
#         env = self.MockEnv()
#         partitions = 2
#         expected_number_of_partitions = partitions ** env.action_space.shape[0]
#         result = create_action_partitions(env, partitions)
#         self.assertEqual(len(result), expected_number_of_partitions)

#     def test_hyperrectangle_bounds(self):
#         env = self.MockEnv()
#         partitions = 2
#         result = create_action_partitions(env, partitions)
#         for hyperrectangle in result:
#             for low, high in zip(hyperrectangle.low, hyperrectangle.high):
#                 self.assertGreaterEqual(high, low)

#     def test_invalid_partitions(self):
#         env = self.MockEnv()
#         with self.assertRaises(ValueError):
#             create_action_partitions(env, -1)

#     def test_single_partition(self):
#         env = self.MockEnv()
#         partitions = 1
#         result = create_action_partitions(env, partitions)
#         self.assertEqual(len(result), 1)

#     def test_large_number_of_partitions(self):
#         env = self.MockEnv()
#         partitions = 10
#         result = create_action_partitions(env, partitions)
#         self.assertEqual(len(result), partitions ** env.action_space.shape[0])

# if __name__ == '__main__':
#     unittest.main()


E
ERROR: /Users/koentuin/Library/Jupyter/runtime/kernel-5856537a-b22f-416a-ab1a-8b4bda031544 (unittest.loader._FailedTest./Users/koentuin/Library/Jupyter/runtime/kernel-5856537a-b22f-416a-ab1a-8b4bda031544)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/koentuin/Library/Jupyter/runtime/kernel-5856537a-b22f-416a-ab1a-8b4bda031544'

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)


SystemExit: True

/Users/koentuin/Documents/Studie/Minor Engineering wit AI/CAI/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
def get_lower_bound(model, state, action_partition, epsilon):
    """
    Computes the lower bound of the output for a given state-action pair using CROWN (Convex Relaxation for neural network Output bounNd) method.

    This function applies CROWN, a method for robustness verification of neural networks, to compute the lower bound of the neural network's output for a given state-action pair within a specified perturbation range. The perturbation range is defined by epsilon around the state-action pair.

    Parameters:
    - model (NN): The neural network model for which the bounds are computed.
    - state (Tensor): The current state of the system, represented as a tensor.
    - action (Tensor): The action to be evaluated, represented as a tensor.
    - epsilon (float): The perturbation range around the state-action pair within which the lower bound is computed.

    Returns:
    - A tensor representing the lower bound of the network's output for the given state-action pair within the specified perturbation range. The shape of the tensor matches the concatenated state-action input shape.

    Notes:
    - The method assumes that the input to 'model' is a concatenation of state and action.
    - The lower bound is computed using the CROWN method, which involves a convex relaxation technique to estimate bounds on neural network outputs.
    - HyperRectangle.from_eps is used to define the perturbation range around the input state-action pair.
    """
    state_partition = HyperRectangle.from_eps(state.view(1, -1), epsilon)

    input_bounds = HyperRectangle(
        torch.cat((state_partition.lower, action_partition.lower), dim=1),
        torch.cat((state_partition.upper, action_partition.upper), dim=1)
    )
    crown_bounds = model.crown(input_bounds)

    crown_bounds.lower # (A, b)

    return crown_bounds.lower

In [5]:
def create_bound_matrices(partitions, state, env, boundnet):
    """
    Computes bound matrices for action partitions in a given state using a bound network.

    Parameters:
    - partitions (Iterable): A collection of partitions of the action space, each representing a subset of potential actions.
    - state (Tensor): The current state of the system, represented as a tensor.
    - env: The environment object, containing information about the action space.
    - boundnet (NN): A neural network used to compute linear bounds for a given state and action.

    Returns:
    - A list of tuples. Each tuple contains:
        1. An action partition,
        2. A tensor representing the action-dependent component of the bound for that partition,
        3. A numpy array representing the state vector component of the bound.
    """
    action_space = env.action_space
    num_actions = action_space.shape[0]
    res = []
    for action_partition in partitions:
        action = torch.tensor(action_partition.center).view(1, -1)
        (A, b) = get_lower_bound(boundnet, state, action_partition, 0.01)
        h_action_dependent = A[:, -num_actions:]

        # State input region is a hyperrectangle with "radius" 0.01
        state_input_bounds = HyperRectangle.from_eps(state, 0.01)
        # State dependent part of the A matrix
        state_A = A[:, :-num_actions]
        # Make this into a (lower) linear bounds (\underbar{A}_x x + b \leq ...)
        state_linear_bounds = LinearBounds(state_input_bounds, (state_A, b), None)
        # Convert to lower interval bounds (b \leq ...)
        state_interval_bounds = state_linear_bounds.concretize()
        # Select the lower bound
        h_vec = state_interval_bounds.lower

        vecs = (action_partition, h_action_dependent.squeeze().detach().numpy(), h_vec.squeeze().detach().numpy())
        res.append(vecs)

    return res

In [6]:
class InfeasibilityError(Exception):
    """Exception raised if there are no actions that fulfill the safety criterions."""

    def __init__(self, message="No safe action to take"):
        self.message = message
        super().__init__(self.message)

In [8]:
def continuous_cbf(bound_matrices, nominal_action, h_current, alpha):
    """
    Selects a safe action from bound matrices using continuous control barrier functions (CBF).

    Parameters:
    - bound_matrices (Iterable): An iterable of tuples, each containing the action partition, the action-dependent component of the barrier function h, and the state vector component of the bound of h.
    - nominal_action (Tensor): The preferred action in the current state, typically derived from an unconstrained policy.
    - h_current (Tensor): The current value of the barrier function.
    - alpha (float): A scaling factor used in the safety criterion. It scales the current barrier function value to set a threshold for the next state's barrier function value.

    Returns:
    - The selected safe action as a numpy array. If multiple safe actions are available, it returns the one closest to the nominal action.

    Raises:
    - InfeasibilityError: If no safe actions are found, indicating that the current state is infeasible under the given safety constraints.
    """
    safe_actions = []
    for action_partition, h_action_dependent, h_vec in bound_matrices:
        num_actions = nominal_action.shape[0]
        action = cp.Variable(num_actions)

        # Constraints
        action_lower_bound = (action_partition).lower.reshape((-1,))
        action_upper_bound = (action_partition).upper.reshape((-1,))
        constraints = [action_lower_bound <= action, action <= action_upper_bound, h_action_dependent @ action + h_vec >= alpha * h_current]

        # Objective
        objective = cp.Minimize(cp.norm(action - nominal_action, 2))

        # Solve the problem
        problem = cp.Problem(objective, constraints)
        problem.solve()

        if problem.status in [cp.INFEASIBLE, cp.UNBOUNDED]:
            # safe_actions.append((None, None))
            pass
        else:
            safe_actions.append((action.value, objective.value))

    if safe_actions and len(safe_actions) > 1:
      return min(safe_actions, key=lambda x: x[1], default=(None, None))[0]
    elif safe_actions:
      return safe_actions[0]
    else:
      raise InfeasibilityError()

In [9]:
class NNDM(nn.Sequential):
    """
    TODO: write docstring
    TODO: generalise class
    """
    def __init__(self):
        super(NNDM, self).__init__(
            nn.Linear(28, 64),
            nn.Tanh(),
            nn.Linear(64, 24),
        )

    def forward(self, x):
        out = super().forward(x)
        return out + x[:,:24]


class HHead(nn.Sequential):
    """
    TODO: implement h function
    TODO: write docstring
    TODO: generalise class
    """
    def __init__(self):

        super().__init__(
            Pow(2),
            nn.Linear(24, 2)
        )

class CombinedModel(nn.Sequential):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.add_module('nndm', NNDM())
        self.add_module('hhead', HHead())

In [10]:
# Initialise the environment
env = gym.make("BipedalWalker-v3")

# Initialise the barrier function h and the NNDM with the barrier function on top to compute linear bounds
hnet = HHead()
net = CombinedModel()
factory = BoundModelFactory()
boundnet = factory.build(net)

# Generate random state, h values, and nominal action for demonstration
state = torch.rand(1, 24)
h_current = hnet(state).view(-1).detach().numpy()
nominal_action = torch.rand(4)

# Apply the CBF to the nominal action
partitions = create_action_partitions(env, 4)
bound_matrices = create_bound_matrices(partitions, state, env, boundnet)
safe_action = continuous_cbf(bound_matrices, nominal_action, h_current, 0)
print(safe_action)

TypeError: expected Tensor as element 1 in argument 0, but got numpy.ndarray